In [1]:
!pip install -Uqq fastbook
import fastbook
try:
  fastbook.setup_book()
except:
  print("No password entered")

     |████████████████████████████████| 727kB 10.7MB/s 
     |████████████████████████████████| 1.0MB 53.9MB/s 
     |████████████████████████████████| 194kB 52.7MB/s 
     |████████████████████████████████| 51kB 9.6MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 40kB 7.4MB/s 
     |████████████████████████████████| 92kB 14.7MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
     |████████████████████████████████| 51kB 9.1MB/s 
     |████████████████████████████████| 2.6MB 55.5MB/s 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:



No password entered


In [2]:
from fastbook import *
from IPython.display import display, HTML

## NLP introduction

nlp is about guessing the next word. for that the model needs to create its own labels. It uses self supervised learning.

The language model used to classify IMDb was pretrained on Wikipedia. 

why learn in detail ?

One reason, of course, is that it is helpful to understand the foundations of the models that you are using. But there is another very practical reason, which is that you get even better results if you fine-tune the (sequence-based) language model prior to fine-tuning the classification model

We will be finetuning the pretrained language model which was trained on wikipedia articles.

This is called ULMFit approach.

### text preprocessing

we already know how categorical variables can be used as independent variables for neural network

make a list of all possible levels of the variable (vocab)

Replace each level with its index in the vocab

create an embedding matrix for this contatining a row for each level i.e. for each item in the vocab

Use this embedding matrix as the first layer of a neural network.

we do the same thing with text. 

whats new is idea of sequence. first we concatenate all of the document in our dataset into onw big long string and split it into words giving us very long words or tokens. 

Our independent variable will be the entire string exect for the second last and last will be labe;

Our vocab would be mix of common words from wikipedia and new words specific to our corpus would be movie actors

for building embedding matrix: for words in vocabualary of pre trained modelwe will take corresponding row in the embedding matrix of the pretrained model but for new words we won't have anythong, we willjust initialize the corresponding row with a random vector

jargon

tokenisation

Numericalisations - making list of unique words -vocab and convert each word to index to look up in vocab

Language model data loader creation -  LMDDataLoader class for seperating the last token as label

Language model creation - creating a model that handles the input list that are arbitaryily small or big.

### Tokenisation

converting to words
 word based - split a sentence on spaces and apply language specific rules to try to seperate parts of meaning even when there are no spaces

 subword - word is split like occ aa sion

 Character based - individual character

 with fastai word tokenisation is done through exernal API

In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

need to gradb the text file, like get_imagefiles we have get_text_file.

we can also pass folder to restrict the search

In [4]:
!dir {path}

imdb.vocab  README  test  tmp_clas  tmp_lm  train  unsup


In [5]:
files = get_text_files(path, folders=['train', 'test', 'unsup'])

In [6]:
txt = files[0].open().read()
txt[:75]

"Hmmmm, interesting. I'll keep this short on detail, as so many have done su"

fastai by default used spacyTokenizer

In [7]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(toks)

['Hmmmm', ',', 'interesting', '.', 'I', "'ll", 'keep', 'this', 'short', 'on', 'detail', ',', 'as', 'so', 'many', 'have', 'done', 'such', 'a', 'good', 'job', 'of', 'pointing', 'out', 'the', 'myriad', 'problems', 'with', 'this', 'series', 'both', 'of', 'itself', 'and', 'as', 'a', 'laughably', 'awful', 'remake', 'of', 'a', 'good', 'original', 'series', '.', 'Ludicrously', 'similar', 'to', 'the', '(', 'usually', ')', 'teen', '-', 'aged', 'modern', '"', 'role', 'playing', 'game', '"', 'fans', 'who', 'constantly', 'talked', 'of', 'the', 'lack', 'of', '"', 'darkness', '"', 'in', 'some', 'game', 'they', 'were', 'involved', 'in', 'a', 'decade', 'past', ',', 'but', 'themselves', 'had', 'so', 'little', 'experience', 'of', '"', 'DAHKNESSSSSSS', '"', 'that', 'they', "'d", 'have', 'shrieked', 'and', 'run', 'not', 'only', 'from', 'anything', 'remotely', 'like', 'one', 'of', 'the', 'creepy', 'crawlies', '(', 'usually', 'human', 'appearing', ',', 'but', 'horrific', 'in', 'some', 'psychological', 'or', 

In [8]:
first(spacy(['The U.S. dollar $1 is $1.00']))

(#8) ['The','U.S.','dollar','$','1','is','$','1.00']

In [9]:
# some additional functionality is added to Tokenizer class

tkn = Tokenizer(spacy)

print(tkn(txt))

# xxbos indicates beginning of the stream

['xxbos', 'h', 'xxrep', '4', 'm', ',', 'interesting', '.', 'xxmaj', 'i', "'ll", 'keep', 'this', 'short', 'on', 'detail', ',', 'as', 'so', 'many', 'have', 'done', 'such', 'a', 'good', 'job', 'of', 'pointing', 'out', 'the', 'myriad', 'problems', 'with', 'this', 'series', 'both', 'of', 'itself', 'and', 'as', 'a', 'laughably', 'awful', 'remake', 'of', 'a', 'good', 'original', 'series', '.', 'xxmaj', 'ludicrously', 'similar', 'to', 'the', '(', 'usually', ')', 'teen', '-', 'aged', 'modern', '"', 'role', 'playing', 'game', '"', 'fans', 'who', 'constantly', 'talked', 'of', 'the', 'lack', 'of', '"', 'darkness', '"', 'in', 'some', 'game', 'they', 'were', 'involved', 'in', 'a', 'decade', 'past', ',', 'but', 'themselves', 'had', 'so', 'little', 'experience', 'of', '"', 'dahkne', 'xxrep', '7', 's', '"', 'that', 'they', "'d", 'have', 'shrieked', 'and', 'run', 'not', 'only', 'from', 'anything', 'remotely', 'like', 'one', 'of', 'the', 'creepy', 'crawlies', '(', 'usually', 'human', 'appearing', ',', 'b

In [10]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html>,
 <function fastai.text.core.replace_rep>,
 <function fastai.text.core.replace_wrep>,
 <function fastai.text.core.spec_add_spaces>,
 <function fastai.text.core.rm_useless_spaces>,
 <function fastai.text.core.replace_all_caps>,
 <function fastai.text.core.replace_maj>,
 <function fastai.text.core.lowercase>]

In [11]:
#??replace_rep

In [12]:
coll_repr(tkn('&copy; Fast.ai ww.fast.ai/INDEX'), 31)

"(#8) ['xxbos','©','xxmaj','fast.ai','ww.fast.ai','/','xxup','index']"

### subword tokenisation

Used especially in language like chinese as they have no spaces

then we 
1. analyse the corpus for most commonly occuring grouping
2. tokenize using the vocab of subword units

In [13]:
txts = L(o.open().read() for o in files[:2000])
# L behaves like a list of items with extra functionalities

In [14]:
txts

(#2000) ['Hmmmm, interesting. I\'ll keep this short on detail, as so many have done such a good job of pointing out the myriad problems with this series both of itself and as a laughably awful remake of a good original series. Ludicrously similar to the (usually) teen-aged modern "role playing game" fans who constantly talked of the lack of "darkness" in some game they were involved in a decade past, but themselves had so little experience of "DAHKNESSSSSSS" that they\'d have shrieked and run not only from anything remotely like one of the creepy crawlies (usually human appearing, but horrific in some psychological or spiritual way) they wanted featured and emphasized into the ground but also shielded and run or simply self-destructed from any realistic darkness that had presented itself into their unthinking little lives. Or perhaps the equally silly conviction of the teen-aged "goths"; kids asserting the world had already gone to hell in its very own handbasket and wearing black and 

we instantiate the tokeniser passing in the size of vocab and then find the common sequences of characters to create the vocab. this is done with setup. it is called automatically but we are doing ti here

In [15]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txts]))[:40])



In [16]:
sp = SubwordTokenizer(vocab_sz=1000)
sp.setup(txts)
print(sp([txts]))
for i in str(sp([txts])):
  print(str(i))

<generator object SentencePieceTokenizer.__call__ at 0x7f502e5e55c8>
<
g
e
n
e
r
a
t
o
r
 
o
b
j
e
c
t
 
S
e
n
t
e
n
c
e
P
i
e
c
e
T
o
k
e
n
i
z
e
r
.
_
_
c
a
l
l
_
_
 
a
t
 
0
x
7
f
5
0
2
e
5
e
5
5
c
8
>


In [17]:
#subword(1000)

If er use samller vocab then each token would be smaller bbut it will take more tokens to represent a sentence.

I its large then most common phrases themselves would be in vocab

subwords are important because then we can also train on genomic sequence and MIDI music data

### Numericalisation

IN this we map the token to integers. It is similar to steps needded to create a categorical model.

in this we make a list of all possible levels of that categorical variable

Replace each level with index in vocab

In [18]:
toks = tkn(txt)

print(coll_repr(tkn(txt), 5))

(#1033) ['xxbos','h','xxrep','4','m'...]


in orderto use numericalise we need to call setup

In [19]:
toks200 = txts[:200].map(tkn)
toks200[199]


(#177) ['xxbos','xxmaj','this','is','xxmaj','uma','xxmaj','thurman',"'s",'first'...]

In [20]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)

"(#2240) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

thedefaults to Numericalize are min_freq=2 and max _vocab=60000 this results in fastai replacing all words other than the most common 60 000 with a special unknown word token xxunk.

this is to avoid overly large embedding matrix

after creating num we can use it like a token

In [21]:
nums = num(toks)[:20]
nums

tensor([   2,    0,    5,  362,    0,   10,  215,   11,    8,   19,  306,  307,   20,  257,   33, 1609,   10,   26,   46,  140])

they can be mapped back intooriginal text

In [22]:
num.vocab[20]

'this'

### Putting text into batches for a language Model

two considerations : irregular length and importance of begining when the previous batch left off

In [23]:
stream = "In this chapter, we will go over what has been done previously and try to be the best thing that ever got into the the history of these leaves. To be frank I never expected me to be the kind of guy who would follow through. But what do you kno here we are doing what we do worst. You can't be at your best everytime, however the problem is I am at my worst more time htan I would like. And to make matters worse I rarely learn."
tokens = tkn(stream)
bs, seq_len = 6, 15
d_tokens = np.array( [ tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False, header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,over,what,has,been,done,previously
and,try,to,be,the,best,thing,that,ever,got,into,the,the,history,of
these,leaves,.,xxmaj,to,be,frank,i,never,expected,me,to,be,the,kind
of,guy,who,would,follow,through,.,xxmaj,but,what,do,you,kno,here,we
are,doing,what,we,do,worst,.,xxmaj,you,ca,n't,be,at,your,best
everytime,",",however,the,problem,is,i,am,at,my,worst,more,time,htan,i


we cant simple give the above words to model because of GPU restrictions.

Sowe neeedto devide this array more finely into subarrays of fixed sequence length

In [24]:
# if we want a sequence len of five for 6 batches of length 15

bs, seq_len = 6, 5
for j in range(0,15):
  d_tokens = np.array([tokens[i*15:i*15+j * seq_len] for i in range(bs)])
  df = pd.DataFrame(d_tokens)
  display(HTML(df.to_html(index=False, header=None)))

xxbos,xxmaj,in,this,chapter
and,try,to,be,the
these,leaves,.,xxmaj,to
of,guy,who,would,follow
are,doing,what,we,do
everytime,",",however,the,problem


xxbos,xxmaj,in,this,chapter,",",we,will,go,over
and,try,to,be,the,best,thing,that,ever,got
these,leaves,.,xxmaj,to,be,frank,i,never,expected
of,guy,who,would,follow,through,.,xxmaj,but,what
are,doing,what,we,do,worst,.,xxmaj,you,ca
everytime,",",however,the,problem,is,i,am,at,my


xxbos,xxmaj,in,this,chapter,",",we,will,go,over,what,has,been,done,previously
and,try,to,be,the,best,thing,that,ever,got,into,the,the,history,of
these,leaves,.,xxmaj,to,be,frank,i,never,expected,me,to,be,the,kind
of,guy,who,would,follow,through,.,xxmaj,but,what,do,you,kno,here,we
are,doing,what,we,do,worst,.,xxmaj,you,ca,n't,be,at,your,best
everytime,",",however,the,problem,is,i,am,at,my,worst,more,time,htan,i


xxbos,xxmaj,in,this,chapter,",",we,will,go,over,what,has,been,done,previously,and,try,to,be,the
and,try,to,be,the,best,thing,that,ever,got,into,the,the,history,of,these,leaves,.,xxmaj,to
these,leaves,.,xxmaj,to,be,frank,i,never,expected,me,to,be,the,kind,of,guy,who,would,follow
of,guy,who,would,follow,through,.,xxmaj,but,what,do,you,kno,here,we,are,doing,what,we,do
are,doing,what,we,do,worst,.,xxmaj,you,ca,n't,be,at,your,best,everytime,",",however,the,problem
everytime,",",however,the,problem,is,i,am,at,my,worst,more,time,htan,i,would,like,.,xxmaj,and


xxbos,xxmaj,in,this,chapter,",",we,will,go,over,what,has,been,done,previously,and,try,to,be,the,best,thing,that,ever,got
and,try,to,be,the,best,thing,that,ever,got,into,the,the,history,of,these,leaves,.,xxmaj,to,be,frank,i,never,expected
these,leaves,.,xxmaj,to,be,frank,i,never,expected,me,to,be,the,kind,of,guy,who,would,follow,through,.,xxmaj,but,what
of,guy,who,would,follow,through,.,xxmaj,but,what,do,you,kno,here,we,are,doing,what,we,do,worst,.,xxmaj,you,ca
are,doing,what,we,do,worst,.,xxmaj,you,ca,n't,be,at,your,best,everytime,",",however,the,problem,is,i,am,at,my
everytime,",",however,the,problem,is,i,am,at,my,worst,more,time,htan,i,would,like,.,xxmaj,and,to,make,matters,worse,i


...

"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, over, what, has, been, done, previously, and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow]"
"[and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do]"
"[these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem]"
"[of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and]"
"[are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"


"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, over, what, has, been, done, previously, and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what]"
"[and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca]"
"[these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my]"
"[of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i]"
"[are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"


"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, over, what, has, been, done, previously, and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we]"
"[and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best]"
"[these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i]"
"[of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"


"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, over, what, has, been, done, previously, and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do]"
"[and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem]"
"[these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and]"
"[of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"


"[xxbos, xxmaj, in, this, chapter, ,, we, will, go, over, what, has, been, done, previously, and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca]"
"[and, try, to, be, the, best, thing, that, ever, got, into, the, the, history, of, these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my]"
"[these, leaves, ., xxmaj, to, be, frank, i, never, expected, me, to, be, the, kind, of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i]"
"[of, guy, who, would, follow, through, ., xxmaj, but, what, do, you, kno, here, we, are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[are, doing, what, we, do, worst, ., xxmaj, you, ca, n't, be, at, your, best, everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"
"[everytime, ,, however, the, problem, is, i, am, at, my, worst, more, time, htan, i, would, like, ., xxmaj, and, to, make, matters, worse, i, rarely, learn, .]"


we need to transform texts inot a stream by concatenating them together. As with images it is best to randomize order of documents (not the text!)

Then we cut the stream to batch which is our batch size. for instance if stream is 50000 then we set a batch size of 10 this will give us 10 mini streams of 5000 tokens

this is the behindthe scenes for fastai librart when we create LMDataLoader. we do it when we apply Numericalize


In [25]:
nums200 = toks200.map(num)

In [26]:
dl = LMDataLoader(nums200)

In [27]:
# grabbing the first batch
x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [28]:
x[0][:20]

tensor([   2,    0,    5,  362,    0,   10,  215,   11,    8,   19,  306,  307,   20,  257,   33, 1609,   10,   26,   46,  140])

In [29]:
' '.join([num.vocab[o] for o in x[0][:20]])

"xxbos xxunk xxrep 4 xxunk , interesting . xxmaj i 'll keep this short on detail , as so many"

### Training a Text classifier

we need to finetune our model pretrained on wikipedia to the corpus of IMDb review and then use that model to train a calssifier

### Language model using datablock

fastai handles Tokenize and Numericalize when TextBlock is passed to datablock
All of the arguments that can be passed to Tokenize and Numericalize can also be passed to TextBlock

there is umary method that is very useful for debugging data issues





In [30]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks= TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

One thing that's different to previous types we've used in `DataBlock` is that we're not just using the class directly (i.e., `TextBlock(...)`, but instead are calling a *class method*. 

The reason that `TextBlock` is special is that setting up the numericalizer's vocab can take a long time (we have to read and tokenize every document to get the vocab). To be as efficient as possible it performs a few optimizations: 

- It saves the tokenized documents in a temporary folder, so it doesn't have to tokenize them more than once
- It runs multiple tokenization processes in parallel, to take advantage of your computer's CPUs

We need to tell `TextBlock` how to access the texts, so that it can do this initial preprocessing—that's what `from_folder` does.

`show_batch` then works in the usual way:

In [31]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj as a massive fan of xxup dm , it goes without saying that i have seen this film numerous times . xxmaj however , i watch it purely for the concert footage … the rest of the film is , um , pretty dreadful , sad to say . \n\n xxmaj famed rock music film director xxup da xxmaj pennebaker followed xxmaj mode around on their late 80s xxmaj music xxmaj for xxmaj the xxmaj masses tour ,","xxmaj as a massive fan of xxup dm , it goes without saying that i have seen this film numerous times . xxmaj however , i watch it purely for the concert footage … the rest of the film is , um , pretty dreadful , sad to say . \n\n xxmaj famed rock music film director xxup da xxmaj pennebaker followed xxmaj mode around on their late 80s xxmaj music xxmaj for xxmaj the xxmaj masses tour , which"
1,it sounds ridiculous but the lead actor does his best to play a modern xxmaj james dean type role where you do care about whether he will recover his piano ability . \n\n xxmaj there are nice cameos from this business partner 's wife as well as his piano teacher . \n\n a couple of the scenes do n't quite run as smoothly as most of the film but you definitely follow the path of this moody wanna - be,sounds ridiculous but the lead actor does his best to play a modern xxmaj james dean type role where you do care about whether he will recover his piano ability . \n\n xxmaj there are nice cameos from this business partner 's wife as well as his piano teacher . \n\n a couple of the scenes do n't quite run as smoothly as most of the film but you definitely follow the path of this moody wanna - be musician


### Fine tuning the language model

to conver integer word indices inot activation that we can use, we will use embeddings, just like we did for collaborative filtering and tabular modelling. We will feed those embeddings into a recurrent neural network using an architechtire called AWD-LSTM

In [32]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, metrics=[accuracy, Perplexity()]
).to_fp16()

In [33]:
learn.save('1epoch')

Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [34]:
learn.load('1epoch')

### fine tuning after unfreezing




In [35]:
learn.unfreeze()
learn.fit_one_cycle(1, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.970833,3.782441,0.323944,43.923122,22:40


In [36]:
learn.save_encoder('finetuned')

### Text Generation

using the model to create random reviews

In [37]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]

In [38]:
print("\n".join(preds))

i liked this movie because it begins with Neil Simon and Chris Rock ( james Earl Jones ) who manage to steal the Pacific Station . It 's the same movie the plot is about .
i liked this movie because it looks like a documentary to us about a young woman who 's pals find out that the girl is a virgin , and she goes to " variable " to see what happens to them . The reason


# Creating the Classifier DataLoader

We're now moving from language model fine-tuning to classifier fine-tuning. To recap, a language model predicts the next word of a document, so it doesn't need any external labels. A classifier, however, predicts some external label—in the case of IMDb, it's the sentiment of a document.

This means that the structure of our DataBlock for NLP classification will look very familiar. It's actually nearly the same as we've seen for the many image classification datasets we've worked with:


In [39]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab), CategoryBlock),
    get_y = parent_label,
    get_items = partial(get_text_files,folders=['train', 'test']),
    splitter = GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [40]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,neg
2,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,pos


The only difference in the above datablock is that
> is_lm is True is not present in TextBlock from_folder

> we pass the vocab that we created forthe langage model fine -tuning so that we have same correspondence to index.

There is achallenge with collating multiple documents 


In [43]:
print(num)

Numericalize:
encodes: (object,object) -> encodes
decodes: (object,object) -> decodes



In [41]:


nums_samp = toks200[:10].map(num)
#numericalisation


In [42]:
nums_samp.map(len)

(#10) [1033,212,128,398,61,426,945,218,265,129]

Pytorch DataLoaders need to collate all the items in a batch into a single tensor and a single tensor has a fixed shape 

In images we used croppping padding etc.
we cant squish so we pad

sorting and padding s automaticalling doene when using a textblock with is_lm =false

because in language model we concatenate data first then split them into equally size sections



In [44]:
learn = text_classifier_learner(dls_clas, AWD_LSTM,drop_mult=0.5, metrics=accuracy).to_fp16()

final step is to use load_encoder instead of load we only have pretrained wieghts available for the encoder, load by default raised an exeption if an incomplete model is loaded


In [45]:
learn = learn.load_encoder('finetuned')

### fine tuning the classifier

In [47]:
learn.fit_one_cycle(1,2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.404233,0.235010,0.904760,01:11


we can pass -2 to the freeze_to to freaaze all exept the last two parameter groups

In [48]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4), 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.291886,0.203919,0.918920,01:17


we will unfreeze a bit more and continue training

In [49]:
learn.freeze_to(-3)
learn.fit_one_cycle(1,slice(5e-3/(2.6**4), 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.225767,0.178110,0.931120,01:37


finally the whole model

In [51]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.194780,0.174615,0.932480,01:59
1,0.176370,0.171268,0.934240,01:59


In [53]:
learn.show_results(max_n=3)

,text,category,category_
0,"xxbos xxmaj there 's a sign on xxmaj the xxmaj lost xxmaj highway that says : \n\n * major xxup spoilers xxup ahead * \n\n ( but you already knew that , did n't you ? ) \n\n xxmaj since there 's a great deal of people that apparently did not get the point of this movie , xxmaj i 'd like to contribute my interpretation of why the plot makes perfect sense . xxmaj as others have pointed out , one single viewing of this movie is not sufficient . xxmaj if you have the xxup dvd of xxup md , you can "" cheat "" by looking at xxmaj david xxmaj lynch 's "" top 10 xxmaj hints to xxmaj unlocking xxup md "" ( but only upon second or third viewing , please . ) ;) \n\n xxmaj first of all , xxmaj mulholland xxmaj drive is",pos,pos
1,"xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxbos xxmaj back in the mid / late 80s , an xxup oav anime by title of "" bubblegum xxmaj crisis "" ( which i think is a military slang term for when technical equipment goes haywire ) made its debut on video , taking inspiration from "" blade xxmaj runner "" , "" the xxmaj terminator "" and maybe even "" robocop "" , with a little dash of xxmaj batman / xxmaj bruce xxmaj wayne -",pos,pos
2,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,pos,pos


## disinformation 

Its easy to spread misinformation madlibs style.



In [62]:
learn.predict('I never likedb the movie at all')

('pos', tensor(1), tensor([0.0147, 0.9853]))

How do you do predictions with this though.